In [98]:
# copyright @2016 Pankax.com

# Simple Neural Network for Mnist DataSets

import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline
# scipy.ndimage for rotating image arrays
import scipy.ndimage

In [99]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        # activation function is the logit function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [106]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.01

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [107]:
# load the mnist training data CSV file into a list
training_data_file = open("train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_list = training_data_list[1:]
# print (type(training_data_list))
training_data_file.close()

In [108]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 10

for e in range(epochs):
    
    # go through all records in the training data set
    for record in training_data_list:
        
        # split the record by the ',' commas
        all_values = record.split(',')
        
        # scale and shift the inputs
        # for image rgb is 256, so divide by 256 and then to avoid 1 * it by 0.99
        # and to avoid 0 add 0.1 to everything
        # input should only have pixel so take out label column out of it
        # which is first column in data so [1:]
        inputs = (numpy.asfarray(all_values[1:]) / 256.0 * 0.99) + 0.01
        
        # create the target output values matrix by using numpy zeros matrix 
        # and then add 0.01 to all
        targets = numpy.zeros(output_nodes) + 0.01
        
        #, except the desired label place in numpy zeros matrix, andm make it 0.99
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        
        #train the model
        n.train(inputs, targets)
        
        
        ## create rotated variations
        # rotated anticlockwise by x degrees
        inputs_plusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), 10, cval=0.01, order=1, reshape=False)
        n.train(inputs_plusx_img.reshape(784), targets)
        # rotated clockwise by x degrees
        inputs_minusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), 10, cval=0.01, order=1, reshape=False)
        n.train(inputs_minusx_img.reshape(784), targets)
        
        # rotated anticlockwise by 10 degrees
        #inputs_plus10_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), 10, cval=0.01, order=1, reshape=False)
        #n.train(inputs_plus10_img.reshape(784), targets)
        # rotated clockwise by 10 degrees
        #inputs_minus10_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), -10, cval=0.01, order=1, reshape=False)
        #n.train(inputs_minus10_img.reshape(784), targets)
        
        
        pass
    pass

print ("Done Yes!")

Done Yes!


In [109]:
# load the mnist test data CSV file into a list
test_data_file = open("test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_list = test_data_list[1:]
test_data_file.close()

In [110]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []
test_file_output = []
ImageIdIndex = 0

# go through all the records in the test data set
for record in test_data_list:
    
    # append empty list for every record
    test_file_output.append([])
    
    # add ImageId to empyt list
    test_file_output[ImageIdIndex].append(ImageIdIndex+1)
    
    # split the record by the ',' commas
    all_values = record.split(',')
    
    # correct answer is first value
    #correct_label = int(all_values[])
    
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[0:]) / 256.0 * 0.99) + 0.01
    
    # query the network
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    
    # add label to corresponding ImageId list
    test_file_output[ImageIdIndex].append(label)
    
    #increment ImageId index
    ImageIdIndex += 1
#     # append correct or incorrect to list
#     if (label == correct_label):
#         # network's answer matches correct answer, add 1 to scorecard
#         scorecard.append(1)
#     else:
#         # network's answer doesn't match correct answer, add 0 to scorecard
#         scorecard.append(0)
#         pass
    
    pass


print (test_file_output[0:1])

[[1, 2]]


In [111]:
import csv
with open('output9.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(['ImageId', 'Label'])
    writer.writerows(test_file_output)